This notebook has been modified from the original at https://github.com/wagonhelm/TF_ObjectDetection_API/blob/master/ChessObjectDetection.ipynb to accomodate my project to detect approaching cars in video frames.

In [ ]:
import skimage
import numpy as np
from skimage import io, transform
import os
import shutil
import glob
import pandas as pd
import xml.etree.ElementTree as ET
import tensorflow as tf
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
import urllib.request
import urllib.error
import scipy as scp
from moviepy.editor import VideoFileClip
from utils import label_map_util
from utils import visualization_utils as vis_util

%matplotlib inline

Before opening the Jupyter Notebook make sure you have cloned the `models` folder into the repository root directory and run the following from the root diretory to install the TensorFlow API

```bash
git clone https://github.com/tensorflow/models.git
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/slim
cd ..
cd ..
```

Set Up Path Directories
--------------------------

In [ ]:
root = os.getcwd()
imagePath = os.path.join(root, 'images')
labelsPath = os.path.join(root, 'labels')
linksPath = os.path.join(imagePath, 'imageLinks')
trainPath = os.path.join(imagePath, 'train')
testPath = os.path.join(imagePath, 'test')

Create Dataset
----------------

Some very large detection datasets such as [Pascal](http://www.cs.stanford.edu/~roozbeh/pascal-context/) and [COCO](http://cocodataset.org/#home) exist already but if you want to train a custom object detection class you simply need to create and label your own dataset.

For detecting approaching cars I collected video from beind my bicycle during some of my bike rides.  I split the video out into frames using the following ffmpeg command:

```bash
ffmpeg -i CapturedVid_640x360.mp4 -ss 26 -vf fps=6 -t 2 data/new/Image_%03d.jpg
```

I manually labeled each frame with bounding boxes and class names to identify cars that are approaching in the frame (class 1) and cars that are not approaching (class 2).


Convert XML Labels to CSV
-----------------------------

## Modified From:
 https://github.comr/datitran/raccoon_dataset/blob/master/xml_to_csv.py

#### I used from command line:
```bash
xml_to_csv.py and json_to_csv.py 
```
### to create csv files

os.chdir(root)

def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (root.find('filename').text,
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text),
                     member[0].text,
                     int(member[4][0].text),
                     int(member[4][1].text),
                     int(member[4][2].text),
                     int(member[4][3].text)
                     )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df


def main():
    
    for i in [trainPath, testPath]:
        image_path = i
        folder = os.path.basename(os.path.normpath(i))
        xml_df = xml_to_csv(image_path)
        xml_df.to_csv('data/'+folder+'.csv', index=None)
        print('Successfully converted xml to csv.')
    
main()

Create TF Record
------------------------------------------------------

When training models with TensorFlow using [tfrecords](http://goo.gl/oEyYyR) files help optimize your data feed.  We can generate a tfrecord using code adapted from this [raccoon detector](https://github.com/datitran/raccoon_dataset/blob/master/generate_tfrecord.py).

### I used generate_tfrecord.py from command line to create test and train records

```bash 
ls ima
python generate_tfrecord.py
mv test.record data
mv train.record data

```

Download Model
----------------

There are [models](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) in the TensorFlow API that you can use depending on your needs.  If you want a high speed model that can work on detecting video feed at high fps the [single shot detection](http://www.cs.unc.edu/%7Ewliu/papers/ssd.pdf) model works best, but you gain speed at the cost of accuracy. Some object detection models detect objects by sliding different sized boxes across the image running the classifier many time on different sections of the image, this of course can be very resource consuming.  As it’s name suggests single shot detection determines all bounding box probabilities in one go, hence why it is a vastly faster model. I’ve already configured the [config](https://github.com/tensorflow/models/tree/master/research/object_detection/samples/configs) file for mobilenet and included it in the GitHub repository for this post.  Depending on your computer you may have to lower the batch size in the config file if you run out of memory.



from command line execute the following:
```bash

wget http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v1_coco_11_06_2017.tar.gz
tar xvzf ssd_mobilenet_v1_coco_11_06_2017.tar.gz
```

Train Model
-------------
Since we are only retraining the last layer of our mobilenet model a high end gpu is not required (but certainly can speed things up). Training time should roughly take an hour.  It will be much easier to watch the training process if you copy and paste the following code into a new terminal in the repository root directory.  Once our loss drops to a consistant level for a good while we can stop TensorFlow training by pressing ctrl+c.

To train the model copy and paste the following code into a new terminal from the repository root directory.  If using Docker create a new terminal pressing `ctrl` + `b` then `c`.

```bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/slim
cd ..
cd ..

python3 models/research/object_detection/train.py --logtostderr --train_dir=data/ --pipeline_config_path=data/ssd_mobilenet_v1_pets.config
```

Watch Training in TensorBoard
---------------------------------

We can use TensorBoard to monitor our total loss and other variables.  From the repository root directory run this command.

```bash
tensorboard --logdir='data'
```

Copy Object Detection Utilities to Our Root Directory
-------------------------------------------------------------

We just need to move some of the utilities from the Object Detection folder into our root path.

In [ ]:
%%bash

cp -R models/research/object_detection/utils/. utils

Export Inference Graph
-------------------------

I highly recommend you expiriment with different checkpoints as your model trains.  We can get a list of all the ckpt files with the following.

In [ ]:
%%bash 
cd data
ls model*.index

You can then added the cpkt number to our trained_checkpoint argument.

In [ ]:
%%bash
export PYTHONPATH="$PYTHONPATH:models/research/slim"

In [ ]:
%%bash 
rm -rf object_detection_graph
python3 models/research/object_detection/export_inference_graph.py \
    --input_type image_tensor \
    --pipeline_config_path data/ssd_mobilenet_v1_pets.config \
    --trained_checkpoint_prefix data/model.ckpt-11449 \
    --output_directory object_detection_graph

Test Model with a single image
-----------

In [ ]:
# Modified From API
# https://github.com/tensorflow/models/blob/master/research/object_detection/object_detection_tutorial.ipynb

from utils import label_map_util
from utils import visualization_utils as vis_util


# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = 'object_detection_graph/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = 'data/label_map.pbtxt'

NUM_CLASSES = 2

PATH_TO_TEST_IMAGES_DIR = '~/data/bike/training/'
TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'image_000012_006.jpg'.format(i)) for i in range(6, 7) ]
IMAGE_SIZE = (360,360)

detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)




In [ ]:

# Modified From API
# https://github.com/tensorflow/models/blob/master/research/object_detection/object_detection_tutorial.ipynb
output_name = "out.jpg"
# with detection_graph.as_default():
with tf.Session(graph=detection_graph) as sess:
    # Definite input and output Tensors for detection_graph
    image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
    # Each box represents a part of the image where a particular object was detected.
    detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
    # Each score represent how level of confidence for each of the objects.
    # Score is shown on the result image, together with the class label.
    detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
    detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
    num_detections = detection_graph.get_tensor_by_name('num_detections:0')
    for image_path in TEST_IMAGE_PATHS:
      image = Image.open(image_path)
      # the array based representation of the image will be used later in order to prepare the
      # result image with boxes and labels on it.
      image_np = load_image_into_numpy_array(image)
      # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
      image_np_expanded = np.expand_dims(image_np, axis=0)
      # Actual detection.
      (boxes, scores, classes, num) = sess.run(
          [detection_boxes, detection_scores, detection_classes, num_detections],
          feed_dict={image_tensor: image_np_expanded})
      # Visualization of the results of a detection.
      visualize_boxes_and_labels_on_image_array(
          image_np,
          np.squeeze(boxes),
          np.squeeze(classes).astype(np.int32),
          np.squeeze(scores),
          category_index,
          use_normalized_coordinates=True,
          line_thickness=2)
      plt.figure(figsize=IMAGE_SIZE)
      plt.imshow(image_np)
        # save Image
      #scp.misc.imsave(output_name, image_np)



## My monkey patching of visualization_utils.py
In the following code I draw a progress bar which visualizes how close the approaching car is based on the size of the box.  I also filter out the bounding boxes that are too large or too small or boxes that are too far in the corners of the screen.

In [ ]:

def draw_bounding_box_on_image(image,
                               ymin,
                               xmin,
                               ymax,
                               xmax,
                               color='red',
                               thickness=4,
                               display_str_list=(),
                               use_normalized_coordinates=True):
  """Adds a bounding box to an image.

  Each string in display_str_list is displayed on a separate line above the
  bounding box in black text on a rectangle filled with the input 'color'.
  If the top of the bounding box extends to the edge of the image, the strings
  are displayed below the bounding box.

  Args:
    image: a PIL.Image object.
    ymin: ymin of bounding box.
    xmin: xmin of bounding box.
    ymax: ymax of bounding box.
    xmax: xmax of bounding box.
    color: color to draw bounding box. Default is red.
    thickness: line thickness. Default value is 4.
    display_str_list: list of strings to display in box
                      (each to be shown on its own line).
    use_normalized_coordinates: If True (default), treat coordinates
      ymin, xmin, ymax, xmax as relative to the image.  Otherwise treat
      coordinates as absolute.
  """
  draw = ImageDraw.Draw(image)
  im_width, im_height = image.size
  if use_normalized_coordinates:
    (left, right, top, bottom) = (xmin * im_width, xmax * im_width,
                                  ymin * im_height, ymax * im_height)
  else:
    (left, right, top, bottom) = (xmin, xmax, ymin, ymax)
  draw.line([(left, top), (left, bottom), (right, bottom),
             (right, top), (left, top)], width=thickness, fill=color)
  try:
    font = ImageFont.truetype('arial.ttf', 24)
  except IOError:
    font = ImageFont.load_default()

  # If the total height of the display strings added to the top of the bounding
  # box exceeds the top of the image, stack the strings below the bounding box
  # instead of above.
  display_str_heights = [font.getsize(ds)[1] for ds in display_str_list]
  # Each display_str has a top and bottom margin of 0.05x.
  total_display_str_height = (1 + 2 * 0.05) * sum(display_str_heights)

  if top > total_display_str_height:
    text_bottom = top
  else:
    text_bottom = bottom + total_display_str_height
  # Reverse list and print from bottom to top.
  for display_str in display_str_list[::-1]:
    text_width, text_height = font.getsize(display_str)
    margin = np.ceil(0.05 * text_height)
    draw.rectangle(
        [(left, text_bottom - text_height - 2 * margin), (left + text_width,
                                                          text_bottom)],
        fill=color)
    draw.text(
        (left + margin, text_bottom - text_height - margin),
        display_str,
        fill='black',
        font=font)
    text_bottom -= text_height - 2 * margin
    if (display_str[0:8] == 'oncoming'):
        #Draw progress bar
      max_area = im_width*im_height *0.20
      area_range = [max_area *0.05, max_area *0.15, max_area*0.35, max_area]
      areas = []
      areas.append((right-left) * (bottom-top))
      if max(areas) < area_range[0]:
        fillcolor = 'green'
      elif max(areas) < area_range[1]:
        fillcolor = 'yellow'
      elif max(areas) < area_range[2]:
        fillcolor = 'orange'
      else:
        fillcolor = 'red'

      draw.rectangle([im_width*0.5-50, im_height-20, (im_width*0.5 + (80*max(areas)/max_area), im_height-45)], fill=fillcolor, outline='black')


def draw_bounding_boxes_on_image(image,
                                 boxes,
                                 color='red',
                                 thickness=4,
                                 display_str_list_list=()):
  """Draws bounding boxes on image.

  Args:
    image: a PIL.Image object.
    boxes: a 2 dimensional numpy array of [N, 4]: (ymin, xmin, ymax, xmax).
           The coordinates are in normalized format between [0, 1].
    color: color to draw bounding box. Default is red.
    thickness: line thickness. Default value is 4.
    display_str_list_list: list of list of strings.
                           a list of strings for each bounding box.
                           The reason to pass a list of strings for a
                           bounding box is that it might contain
                           multiple labels.

  Raises:
    ValueError: if boxes is not a [N, 4] array
  """
  boxes_shape = boxes.shape
  if not boxes_shape:
    return
  if len(boxes_shape) != 2 or boxes_shape[1] != 4:
    raise ValueError('Input must be of size [N, 4]')
  for i in range(boxes_shape[0]):
    display_str_list = ()
    if display_str_list_list:
      display_str_list = display_str_list_list[i]
    #If box is too small do not draw
    if(boxes[i, 2] < image.height/6):
        break
    if (boxes[i, 1] > image.width/6):
        break
    #If box is too large, do not draw
    print(boxes[i, 2]-boxes[i, 0])*(box[i, 3]-box[i, 1])
    draw_bounding_box_on_image(image, boxes[i, 0], boxes[i, 1], boxes[i, 2],
                               boxes[i, 3], color, thickness, display_str_list)

def visualize_boxes_and_labels_on_image_array(image,
                                              boxes,
                                              classes,
                                              scores,
                                              category_index,
                                              instance_masks=None,
                                              keypoints=None,
                                              use_normalized_coordinates=False,
                                              max_boxes_to_draw=20,
                                              min_score_thresh=.5,
                                              agnostic_mode=False,
                                              line_thickness=4):
  """Overlay labeled boxes on an image with formatted scores and label names.

  This function groups boxes that correspond to the same location
  and creates a display string for each detection and overlays these
  on the image. Note that this function modifies the image in place, and returns
  that same image.

  Args:
    image: uint8 numpy array with shape (img_height, img_width, 3)
    boxes: a numpy array of shape [N, 4]
    classes: a numpy array of shape [N]. Note that class indices are 1-based,
      and match the keys in the label map.
    scores: a numpy array of shape [N] or None.  If scores=None, then
      this function assumes that the boxes to be plotted are groundtruth
      boxes and plot all boxes as black with no classes or scores.
    category_index: a dict containing category dictionaries (each holding
      category index `id` and category name `name`) keyed by category indices.
    instance_masks: a numpy array of shape [N, image_height, image_width], can
      be None
    keypoints: a numpy array of shape [N, num_keypoints, 2], can
      be None
    use_normalized_coordinates: whether boxes is to be interpreted as
      normalized coordinates or not.
    max_boxes_to_draw: maximum number of boxes to visualize.  If None, draw
      all boxes.
    min_score_thresh: minimum score threshold for a box to be visualized
    agnostic_mode: boolean (default: False) controlling whether to evaluate in
      class-agnostic mode or not.  This mode will display scores but ignore
      classes.
    line_thickness: integer (default: 4) controlling line width of the boxes.

  Returns:
    uint8 numpy array with shape (img_height, img_width, 3) with overlaid boxes.
  """
  # Create a display string (and color) for every box location, group any boxes
  # that correspond to the same location.

  box_to_display_str_map = collections.defaultdict(list)
  box_to_color_map = collections.defaultdict(str)
  box_to_instance_masks_map = {}
  box_to_keypoints_map = collections.defaultdict(list)
  if not max_boxes_to_draw:
    max_boxes_to_draw = boxes.shape[0]
  for i in range(min(max_boxes_to_draw, boxes.shape[0])):
    if scores is None or scores[i] > min_score_thresh:
      box = tuple(boxes[i].tolist())
      if instance_masks is not None:
        box_to_instance_masks_map[box] = instance_masks[i]
      if keypoints is not None:
        box_to_keypoints_map[box].extend(keypoints[i])
      if scores is None:
        box_to_color_map[box] = 'black'
      else:
        if not agnostic_mode:
          if classes[i] in category_index.keys():
            class_name = category_index[classes[i]]['name']
          else:
            class_name = 'N/A'
          display_str = '{}: {}%'.format(
              class_name,
              int(100*scores[i]))
        else:
          display_str = 'score: {}%'.format(int(100 * scores[i]))
        box_to_display_str_map[box].append(display_str)
        if agnostic_mode:
          box_to_color_map[box] = 'DarkOrange'
        else:
          box_to_color_map[box] = STANDARD_COLORS[
              classes[i] % len(STANDARD_COLORS)]

  # Draw all boxes onto image.
  for box, color in box_to_color_map.items():
    ymin, xmin, ymax, xmax = box
    # Do not draw the box if it is too large.
    if(ymax-ymin)*(xmax-xmin) > 0.4:
        break
    if (ymax-ymin) > 0.5:
        break
    if (xmax-xmin) > 0.5:
        break
    if instance_masks is not None:
      draw_mask_on_image_array(
          image,
          box_to_instance_masks_map[box],
          color=color
      )
    draw_bounding_box_on_image_array(
        image,
        ymin,
        xmin,
        ymax,
        xmax,
        color=color,
        thickness=line_thickness,
        display_str_list=box_to_display_str_map[box],
        use_normalized_coordinates=use_normalized_coordinates)
    if keypoints is not None:
      draw_keypoints_on_image_array(
          image,
          box_to_keypoints_map[box],
          color=color,
          radius=line_thickness / 2,
          use_normalized_coordinates=use_normalized_coordinates)

  return image



## Process the videos
In the following code I take an input video, break it into frames, classify each frame and draw bounding boxes, and then glue the frames back together to output an mp4.

In [ ]:
test_videos = [14, 17, 23, 29, 33, 35, 39, 43, 55]
for t in test_videos:
    #del output_clip
    output_name = 'out_'+str(t)+'_hq.mp4'
    input_video = 'test/bike/clip_0000'+str(t)+'_hq.mp4'
    with detection_graph.as_default():
      with tf.Session(graph=detection_graph) as sess:
        def pipeline(image):
            image_np = image
            # Definite input and output Tensors for detection_graph
            image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
            # Each box represents a part of the image where a particular object was detected.
            detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
            # Each score represent how level of confidence for each of the objects.
            # Score is shown on the result image, together with the class label.
            detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
            detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
            num_detections = detection_graph.get_tensor_by_name('num_detections:0')
            #image = Image.open(image)
            # the array based representation of the image will be used later in order to prepare the
            # result image with boxes and labels on it.
            #image_np = load_image_into_numpy_array(image)
            # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
            image_np_expanded = np.expand_dims(image_np, axis=0)
            # Actual detection.
            (boxes, scores, classes, num) = sess.run(
              [detection_boxes, detection_scores, detection_classes, num_detections],
              feed_dict={image_tensor: image_np_expanded})
            # Visualization of the results of a detection.
            #(boxes, scores, classes, num) = reduce_boxes(boxes, scores, classes, num)
            visualize_boxes_and_labels_on_image_array(
              image_np,
              np.squeeze(boxes),
              np.squeeze(classes).astype(np.int32),
              np.squeeze(scores),
              category_index,
              use_normalized_coordinates=True,
              line_thickness=2)
            # save Image
            return image_np
    clip = VideoFileClip(input_video)
    output_clip = clip.fl_image(pipeline)
    output_clip.write_videofile(output_name, audio=False, ffmpeg_params=['-b:v', '4M'])
    #output_clip.write_videofile(output_name, audio=False)
